# 海龜交易策略回測範例

這個筆記本示範如何使用我們的交易策略回測框架來測試海龜交易策略。

In [ ]:
# 導入必要的套件
import os
import sys

sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from backtest_engine import BacktestEngine
from config.config import get_config
from strategies.turtle_strategy import TurtleStrategy
from utils import fetch_data, plot_price_and_signals, print_strategy_summary

## 1. 獲取市場資料

In [ ]:
# 選擇要測試的股票
symbol = 'QQQ'
period = '5y'  # 5年的資料

# 獲取資料
print(f"正在獲取 {symbol} 的資料...")
data = fetch_data(symbol, period=period)
print(f"獲取了 {len(data)} 筆資料")
data.head()

正在獲取 QQQ 的資料...
獲取了 1255 筆資料


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2020-08-27 00:00:00-04:00,284.486978,285.341502,280.350317,282.622559,49482700,0.0,0.0,0.0
2020-08-28 00:00:00-04:00,283.865502,284.690896,282.506045,284.059723,28021000,0.0,0.0,0.0
2020-08-31 00:00:00-04:00,284.584056,288.157493,284.147074,286.341644,36321000,0.0,0.0,0.0
2020-09-01 00:00:00-04:00,289.002328,291.352269,287.225329,291.235748,36216200,0.0,0.0,0.0
2020-09-02 00:00:00-04:00,294.498493,294.712125,288.293532,293.993561,50836800,0.0,0.0,0.0


## 2. 創建交易策略

In [ ]:
# 創建海龜交易策略
strategy = TurtleStrategy(
    name="海龜策略 (20/10)",
    entry_window=20,
    exit_window=10,
    initial_capital=100000
)

print(f"策略名稱: {strategy.name}")
print(f"進場窗口: {strategy.entry_window} 天")
print(f"出場窗口: {strategy.exit_window} 天")
print(f"初始資金: ${strategy.initial_capital:,.2f}")

策略名稱: 海龜策略 (20/10)
進場窗口: 20 天
出場窗口: 10 天
初始資金: $100,000.00


## 3. 運行回測

In [ ]:
# 創建回測引擎
backtest_config = get_config('backtest')
engine = BacktestEngine(
    initial_capital=backtest_config['initial_capital'],
    commission=backtest_config['commission']
)

# 運行回測
print("開始回測...")
result = engine.run_backtest(strategy, data, symbol)
print("回測完成!")

開始回測...
回測完成!


## 4. 分析結果

In [ ]:
# 顯示策略摘要
performance = result['performance']
print_strategy_summary(strategy.name, performance)


策略： 海龜策略 (20/10)
總回報率： 1376.01%
年化回報率： 71.69%
波動率： 35.50%
夏普比率： 2.020
最大回撤： 1.69%
總交易次數： 46

Strategy: 海龜策略 (20/10)
Total Return: 1376.01%
Annualized Return: 71.69%
Volatility: 35.50%
Sharpe Ratio: 2.020
Max Drawdown: 1.69%
Total Trades: 46


In [ ]:
# 顯示交易記錄
trades_df = pd.DataFrame(result['trades'])
if not trades_df.empty:
    print(f"總交易次數: {len(trades_df)}")
    print("前10筆交易:")
    print(trades_df.head(10))
else:
    print("沒有執行任何交易")

總交易次數: 46
前10筆交易:
                  timestamp symbol  quantity       price         value  \
0 2020-10-09 00:00:00-04:00    QQQ       161  277.437195  44667.388367   
1 2020-10-26 00:00:00-04:00    QQQ      -161  272.348938 -43848.179016   
2 2020-12-01 00:00:00-05:00    QQQ       183  294.673218  53925.198853   
3 2021-02-22 00:00:00-05:00    QQQ      -183  313.671143 -57401.819092   
4 2021-04-01 00:00:00-04:00    QQQ       141  316.142090  44576.034668   
5 2021-05-04 00:00:00-04:00    QQQ      -141  321.567444 -45341.009583   
6 2021-06-08 00:00:00-04:00    QQQ       208  328.015533  68227.230957   
7 2021-08-18 00:00:00-04:00    QQQ      -208  353.213959 -73468.503418   
8 2021-08-23 00:00:00-04:00    QQQ       250  363.960266  90990.066528   
9 2021-09-20 00:00:00-04:00    QQQ      -250  357.012634 -89253.158569   

   capital_after  
0   55332.611633  
1   99180.790649  
2   89059.922634  
3  146461.741726  
4  159230.124331  
5  204571.133913  
6  181639.571529  
7  255108.07494

## 5. 視覺化分析

In [ ]:
# 繪製價格和交易信號
signals_data = result['signals_data']
plot_price_and_signals(signals_data, f"{strategy.name} - {symbol}")

In [ ]:
# 繪製投資組合價值變化
portfolio_value = result['portfolio_value']
if portfolio_value:
    plt.figure(figsize=(12, 6))
    
    # 投資組合價值
    portfolio_series = pd.Series(portfolio_value, index=data.index[:len(portfolio_value)])
    plt.plot(portfolio_series.index, portfolio_series.values, label='投資組合價值', linewidth=2)
    
    # 買入持有策略比較
    buy_hold_value = (data['Close'] / data['Close'].iloc[0]) * strategy.initial_capital
    plt.plot(data.index, buy_hold_value, label='買入持有', linewidth=2, alpha=0.7)
    
    plt.title(f'{strategy.name} - 投資組合價值 vs 買入持有')
    plt.xlabel('日期')
    plt.ylabel('價值 ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 計算買入持有的績效
    buy_hold_return = (buy_hold_value.iloc[-1] / strategy.initial_capital) - 1
    strategy_return = performance.get('total_return', 0)
    
    print(f"績效比較:")
    print(f"策略總回報: {strategy_return:.2%}")
    print(f"買入持有回報: {buy_hold_return:.2%}")
    print(f"超額回報: {(strategy_return - buy_hold_return):.2%}")